In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import torch
import tangram as tg
from sklearn.model_selection import train_test_split
from anndata import AnnData, read_h5ad

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_merfish = read_h5ad("data/liver_merfish.h5ad")

In [ ]:
Rdata_ind, Qdata_ind, _, _ =train_test_split(range(data_merfish.shape[0]), data_merfish.obs['louvain'], test_size=0.5,random_state=1,stratify=data_merfish.obs['louvain'])

Rdata = data_merfish[np.sort(Rdata_ind), :]
Qdata = data_merfish[np.sort(Qdata_ind), :]

## Cannot run on the entire dataset, downsampling is needed
Rdata_ind, Qdata_ind, _, _ =train_test_split(range(Rdata.shape[0]), Rdata.obs['louvain'], test_size=0.5,random_state=1,stratify=Rdata.obs['louvain'])
Rdata = Rdata[np.sort(Qdata_ind), :]

Rdata_ind, Qdata_ind, _, _ =train_test_split(range(Qdata.shape[0]), Qdata.obs['louvain'], test_size=0.5,random_state=1,stratify=Qdata.obs['louvain'])
Qdata = Qdata[np.sort(Qdata_ind), :]

print((Rdata.shape, Qdata.shape))

In [5]:
tg.pp_adatas(Qdata, Rdata, genes=Rdata.var.index)

INFO:root:500 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:500 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


In [6]:
ad_map = tg.map_cells_to_space(
    adata_sc=Qdata,
    adata_sp=Rdata,
    device='cpu',
    # device='cuda:0',
)

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 500 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.386, KL reg: 0.056
Score: 0.716, KL reg: 0.003
Score: 0.750, KL reg: 0.002
Score: 0.756, KL reg: 0.002
Score: 0.759, KL reg: 0.001
Score: 0.760, KL reg: 0.001
Score: 0.761, KL reg: 0.001
Score: 0.762, KL reg: 0.001
Score: 0.763, KL reg: 0.001
Score: 0.763, KL reg: 0.001


INFO:root:Saving results..


In [8]:
Rdata_location_pred = ad_map.X.dot(np.array(Rdata.obs[['x_cord', 'y_cord']]))
np.save("output/liver/tangram_liver.npy", Rdata_location_pred)